# Lab 4.2 - Metody inicjalizacji k-means

**Wykonanie rozwiązań: Marcin Przewięźlikowski**

https://github.com/mprzewie/ml_basics_course

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.cluster import KMeans
from sklearn.cluster.k_means_ import _init_centroids
from sklearn.metrics import calinski_harabaz_score
from typing import Dict, Any, Callable
from matplotlib.colors import CSS4_COLORS

## Zbiór normalny
Wygenerujmy zbiór danych wyglądający +- tak jak na załączonym rysunku.

In [ ]:
p_per_cluster = 30
Xs = []

for i in range(3):
    for j in range(3):
        cluster = np.random.randn(p_per_cluster, 2) + np.array([i, j]) * 6
        Xs.append(cluster)

X = np.concatenate(Xs)

In [ ]:
plt.scatter(X[:, 0], X[:, 1])
plt.show()

Uruchamiamy na nim algorytm k-means z k równym 9 i następującymi metodami inicjowania środków klastrów:

In [ ]:
inits = {}

* Random - z rozkładem jednostajnym po całym zakresie wartości;

In [ ]:
inits["random"] = lambda X: np.random.rand(9,2) * (X.max(axis=0) - X.min(axis=0)) + X.min(axis=0)

* Forgy - wybieramy k elementów ze zbioru jako początkowe środki;

In [ ]:
inits["forgy"] = lambda X: _init_centroids(X, 9, "random")

* Random Partition - losowo dzielimy zbiór na k klastrów, początkowy środek klastra to średnia z elementów które w ten sposób w nim się znalazły;

In [ ]:
inits["random_partition"] = lambda X: np.array([
    X[np.random.randint(0, 9, X.shape[0])==i].mean(axis=0)
    for i in range(9)
])

* k-means++ - wybieramy początkowe środki w sposób opisany w paperze z załącznika.

In [ ]:
inits["kmeans++"] = lambda X: _init_centroids(X, 9, "k-means++")

Trenuję k-means:

In [ ]:
kmeanses: Dict[str, KMeans] = {
    init_name: KMeans(n_clusters=9, init=init_method(X), n_init=1).fit(X)
    for (init_name, init_method) in inits.items()
}

In [ ]:
def kmeans_results_chart(X: np.ndarray, kmeanses: Dict[str, KMeans]):
    fig = plt.figure(figsize=(15,15))
    fig.suptitle("Wizualizacja klasteryzacji w zależności od inicjalizacji")
    for (i, (init_name, kmeans)) in enumerate(kmeanses.items()):
        plt.subplot(2, 2, i+1)
        X_clusters = kmeans.predict(X)
        plt.title(init_name)
        for c in np.unique(X_clusters):
            X_clustered = X[X_clusters==c]
            p = plt.scatter(X_clustered[:, 0], X_clustered[:,1])
            centre = kmeans.cluster_centers_[c]
            plt.scatter([centre[0]], [centre[1]], marker="D", c="white", s=300)
            plt.scatter([centre[0]], [centre[1]], marker="D", c=p.get_facecolors(), s=200)
    plt.show()

In [ ]:
kmeans_results_chart(X, kmeanses)

W klastryzacji tak symaetrycznego zbioru punktów przoduje $random\_partition$ oraz $kmeans++$

Naszym celem jest uzyskanie wykresu jakości klastryzacji Q w zależności od numeru iteracji n dla wszystkich powyższych metod (wszystkie wyniki na jednym wykresie). Jakość Q rozumiemy jako wybraną metrykę jakości (np. Davies-Bouldin index czy Dunn index, może być dowolna rozsądna inna - ale nie Silhouette). Proces k-means jest silnie stochastyczny, więc eksperyment powtarzamy wielokrotnie, a na wykresie pokazujemy średni wynik i jego odchylenie standardowe jako errorbary.

Korzystam z metryki Calinskiego-Harabaza

In [ ]:
def kmeans_score(
    init_method: Any, 
    X: np.ndarray, 
    max_iters: int, 
    n_clusters=9, 
) -> float:
    labels = KMeans(
        n_clusters=n_clusters, 
        init=init_method,
        max_iter=max_iters,
        n_init=1,
        tol=0
    ).fit_predict(X)
    return calinski_harabaz_score(X, labels)

In [ ]:
def kmeans_scores_measurements(
    X: np.ndarray, 
    inits: Dict[str, Callable[[np.ndarray], np.ndarray]] = inits,
    max_iters: int = 30, 
    n_trials: int = 15
):
    scores = {}
    for init_name in inits.keys():
        scores[init_name] = {}
        for trial_no in range(n_trials):
            init_method = inits[init_name](X)
            scores[init_name][trial_no] = [
                kmeans_score(init_method, X, iters)
                for iters in range(1, max_iters)
            ]
    return scores

In [ ]:
scores =kmeans_scores_measurements(X)

In [ ]:
def kmeans_scores_chart(scores: Dict):
    fig = plt.figure(figsize=(12,15))
    for (i, init_name) in enumerate(inits.keys()):
        all_trials_results = np.array(list(scores[init_name].values()))
        means = all_trials_results.mean(axis=0)
        stds = all_trials_results.std(axis=0) / 5
        plt.errorbar(
            [i for (i, _) in enumerate(means)], 
            means, 
            yerr=stds, 
            label=init_name,
        )
    
    plt.legend()
    plt.title("Calinsky-Harabaz score of k-menas with different initialization methods \n" + 
              "Errorbars have been descaled 5 times, for the sake of readability of the chart")
    plt.xlabel("num_iters")
    plt.ylabel("calinsky-harabaz_score")
    plt.show()

In [ ]:
kmeans_scores_chart(scores)

$K-means++$ wyraźnie przebija pozostałe inicjalizacje. 

Zauważyć też można, że wykresy dość szybko się wypłaszczają, czyli że niezależnie od inicjalizacji, K-Means zbiega do rozwiązania już po kilkunastu iteracjach.

 ## Zbiór zepsuty
 Następnie "psujemy zbiór", dokonując następujących zmian:

In [ ]:
broken_clusters = {}
p_per_cluster = 60

 * jeden z okręgów znacząco powiększamy

In [ ]:
broken_clusters["big"] = np.random.randn(p_per_cluster, 2) * 4

 * jeden czynimi znacząco gęściej zapełnionym

In [ ]:
broken_clusters["dense"] = np.random.randn(p_per_cluster * 10, 2) * 1.5 + np.array([0, 10])

 * dwa zbliżamy mocno do siebie

In [ ]:
broken_clusters["close1"] = np.random.randn(p_per_cluster, 2) + np.array([12, 0])
broken_clusters["close2"] = np.random.randn(p_per_cluster, 2) + np.array([3,3]) + np.array([12, 0])

 * jednemu zmieniamy kształt z okrągłego na wrzecionowaty

In [ ]:
broken_clusters["huso"] = np.random.randn(p_per_cluster, 2) * np.array([2, 0.5]) + np.array([12,12])

 * jeden znacząco oddalamy od pozostałych

In [ ]:
broken_clusters["far"] = np.random.randn(p_per_cluster, 2) + np.array([30, 30])

In [ ]:
broken_clusters["normal1"] = np.random.randn(p_per_cluster, 2) + np.array([0, 20]) 
broken_clusters["normal2"] = np.random.randn(p_per_cluster, 2) + np.array([20,0]) 
broken_clusters["normal3"] = np.random.randn(p_per_cluster, 2) + np.array([20,10]) 

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
for (name, c) in broken_clusters.items():
    ax.scatter(c[:,0], c[:,1], label=name)
ax.legend()
plt.show()

Powtarzamy obliczenia dla zmodyfikowanego zbioru - jakie efekty teraz uzyskaliśmy? Jaki był stan końcowy?

In [ ]:
X_broken = np.concatenate(list(broken_clusters.values()))

In [ ]:
kmeanses_broken: Dict[str, KMeans] = {
    init_name: KMeans(n_clusters=9, init=init_method(X_broken), n_init=1).fit(X_broken)
    for (init_name, init_method) in inits.items()
}
kmeans_results_chart(X_broken, kmeanses_broken)

Ze sklastrowaniem zbioru zepsutego zdecydowanie najlepiej poradziło sobie $kmeans++$

Finalnie, dla obu zbiorów (oryginalnego i "zepsutego") uruchommy dla różnych k z zakresu od 1 do 20 i porównajmy finalne metryki jakości. 

In [ ]:
fig = plt.figure(figsize=(12,6))
k_range = range(2,21)
for (ds_name, ds) in [("normal dataset", X), ("broken dataset", X_broken)]:
    scores = [
        calinski_harabaz_score(
            ds, KMeans(n_clusters=k).fit_predict(ds)
        ) for k in k_range
    ]
    plt.plot(list(k_range), scores, label=ds_name)
    
plt.legend()
plt.xticks(list(k_range))
plt.xlabel("n_clusters")
plt.ylabel("Calinshy-Harabaz score")
plt.title("Calinsky-Harabaz score for k-means clustering with k++ initialization")
plt.show()

Czy na ich podstawie można stwierdzić, że optymalne k to 9 (bo tyle mamy klastrów)?

Przy niezepsutym datasecie osiągamy maksymalną wartość punktacji C-H dla $K=9$, więc faktycznie można stwierdzić, że optymalne K=9.

Przy datasecie niezepsutym, maksimum punktacji C-H jest dla $K=17$, czyli dla klasteryzacji wyglądającej tak:

In [ ]:
kmeans = KMeans(n_clusters=17).fit(X_broken)

X_clusters = kmeans.predict(X_broken)
fig = plt.figure(figsize=(10,10))
plt.title("Klasteryzacja zepsutego datasety dla $K=17$")
for c in np.unique(X_clusters):
    X_clustered = X_broken[X_clusters==c]
    p = plt.scatter(X_clustered[:, 0], X_clustered[:,1])

    centre = kmeans.cluster_centers_[c]
    plt.scatter([centre[0]], [centre[1]], marker="D", c="white", s=300)
    plt.scatter([centre[0]], [centre[1]], marker="D", c=p.get_facecolors(), s=200)

plt.show()

Był to jednak dataset trudny do sklastrowania, więc można było się spodziewać, że $K=9$ nie okaże się optymalne.